In [3]:
import os
import sys
import random
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# pytorch
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

# augmentation
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

# sci-kit learn
from sklearn.model_selection import StratifiedKFold

# etc
import time


from effdet.config.model_config import efficientdet_model_param_dict
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain
from effdet.efficientdet import HeadNet
from effdet.config.model_config import efficientdet_model_param_dict

import timm
import gc

sys.path.insert(0, '/home/kerrykim/PycharmProjects/5.skku/wbf')
from wbf.ensemble_boxes import *
import warnings
warnings.filterwarnings("ignore")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
## Configurations
class CFG:
    mode = 'test'
    seed = 42
    print_freq = 30

    n_class = 1

    img_size = 512

    num_fold = 5
    num_epoch = 50
    batch_size = 8
    num_workers = 8    # decide how many data upload to dataset for a batch
                       # if n_workers 2, dataloader works twice for a batch.
                       # It has impact for cuda memory too

    lr = 0.0002

    max_grad_norm = 1000

    data_dir = './data/testing_images'
    ckpt_dir = './trained_effdet'
    result_dir = './result'
    log_dir = './log'

    csv_dir = './df.csv'
    csv2_dir = './df_kfold.csv'
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
class Dataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform

        lst_input = os.listdir(self.data_dir)
        self.lst_input = lst_input
        self.image_id = lst_input
        # [image_id.split('/')[-1][:-4] for image_id in lst_input]    # image_id.jpg에서 .jpg 삭제

    def __len__(self):
        return len(self.lst_input)

    def __getitem__(self, index):
        image_id = self.image_id[index]
        image = cv2.imread(os.path.join(self.data_dir, self.lst_input[index]), cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        if self.transform:
            sample = {'image' : image}
            sample = self.transform(**sample)    # 이때까지 해왔던 입력방식은 cv2 이미지를 직접 입력, 여기선 numpy 로 입력
            image = sample['image']

        target = {}
        target['boxes'] = torch.tensor([0.0, 0.0, 0.0, 0.0])
        target['img_scale'] = torch.tensor([1.])
        target['img_class'] = torch.tensor([1.])
        target['img_size'] = torch.tensor([(512, 512)])

            
        return image, target, image_id

In [6]:
def create_model(num_classes=1, image_size=512, architecture="tf_efficientnetv2_l"):
    efficientdet_model_param_dict['tf_efficientnetv2_l'] = dict(
        name='tf_efficientnetv2_l',
        backbone_name='tf_efficientnetv2_l',
        backbone_args=dict(drop_path_rate=0.2),
        num_classes=num_classes,
        url='', )

    config = get_efficientdet_config(architecture)
    config.update({'num_classes': num_classes})
    config.update({'image_size': (image_size, image_size)})

    # print(config)

    net = EfficientDet(config, pretrained_backbone=True)
    net.class_net = HeadNet(
        config,
        num_outputs=config.num_classes,
    )
    return DetBenchTrain(net, config)

In [7]:
def create_model(num_classes=1, image_size=512, architecture="tf_efficientnetv2_l"):
    efficientdet_model_param_dict['tf_efficientnetv2_l'] = dict(
        name='tf_efficientnetv2_l',
        backbone_name='tf_efficientnetv2_l',
        backbone_args=dict(drop_path_rate=0.2),
        num_classes=num_classes,
        url='', )

    config = get_efficientdet_config(architecture)
    config.update({'num_classes': num_classes})
    config.update({'image_size': (image_size, image_size)})

    # print(config)

    net = EfficientDet(config, pretrained_backbone=True)
    net.class_net = HeadNet(
        config,
        num_outputs=config.num_classes,
    )
    return DetBenchTrain(net, config)

In [8]:
def load_net(ckpt_path):
    net = create_model(num_classes=1, image_size=512, architecture='tf_efficientnetv2_l')

    checkpoint = torch.load(ckpt_path)
    net.load_state_dict(checkpoint)
    
    del checkpoint
    gc.collect()

    return net

In [9]:
def transform_test():
    return A.Compose([A.Resize(height=512, width=512, p=1.0), ToTensorV2(p=1.0)], p=1.0)

In [10]:
def test_one_epoch(loader_test, net, score_threshold=0.22):
    net.eval()
    pred = []

    ##
    for batch, (images, targets, image_ids) in enumerate(loader_test, 1):
        images = torch.stack(images)
        images = images.to(device).float()
        
        images_tensor = images
    
        if images_tensor.ndim == 3:
            images_tensor = images_tensor.unsqueeze(0)
        if (images_tensor.shape[-1] != 512 or images_tensor.shape[-2] != 512):
            raise ValueError(
                f"Input tensors must be of shape (N, 3, {512}, {512})")

        num_images = images_tensor.shape[0]

    
        # 이건 리스케일시 사용
        image_sizes = [(512, 512)] * num_images

        dummy = create_dummy_inference_targets(num_images)
            
#         dummy_targets = create_dummy_inference_targets(images.shape[0])

#         img_scale = torch.tensor([torch.tensor([1.]) for target in targets]).to(device) # 여긴 float이 없다?
#         img_size = torch.tensor([(512, 512) for target in targets]).to(device).float()

#         target_res = {}
#         target_res['img_scale'] = img_scale
#         target_res['img_size'] = img_size
        
        with torch.no_grad():
            det = net(images, dummy)["detections"]
            
            output = []
            for i in range(images.shape[0]):
                boxes = det[i].detach().cpu().numpy()[:,:4]
                scores = det[i].detach().cpu().numpy()[:,4]
                print(scores)
                classes = det[i].detach().cpu().numpy()[:, 5]
                # print(classes)
                indexes = np.where(scores > score_threshold)[0]
                
                boxes = boxes[indexes]
#                 boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
#                 boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
                output.append({"boxes": boxes, "scores": scores[indexes], "classes": classes[indexes]})

    return output
'''

        for i, image in enumerate(images):
            # predicted outputs
#             boxes, scores, labels = run_wbf(output, image_index=i)
            boxes, scores, labels = run_wbf(output, image_size=CFG.img_size, iou_thr=0.44)

            print(boxes)
            print(scores)
            print(labels)
            
            image_id = image_ids[i]
            
            # transform 과정에서 리사이즈된 사진을 원래 크기로 원복함
            scaled_bboxes = []
            img_sizes = [(CFG.img_size, CFG.img_size)] * images.shape[0]       
            
            
#             for bboxes, img_dims in zip(boxes, img_sizes):
                
#                 im_h, im_w = img_dims
                
                
#                 if len(bboxes) > 0:
#                     scaled_bboxes.append((np.array(bboxes)*[im_w / CFG.img_size, im_h / CFG.img_size, 
#                                                             im_w / CFG.img_size, im_h / CFG.img_size,]).tolist())
#                 else:
#                     scaled_bboxes.append(bboxes)
            

     
            
#             tmp = {'image_id': image_id, 'PredictionString': format_prediction_string(boxes, scores)}

#             pred.append(tmp)

    return pred
'''

"\n\n        for i, image in enumerate(images):\n            # predicted outputs\n#             boxes, scores, labels = run_wbf(output, image_index=i)\n            boxes, scores, labels = run_wbf(output, image_size=CFG.img_size, iou_thr=0.44)\n\n            print(boxes)\n            print(scores)\n            print(labels)\n            \n            image_id = image_ids[i]\n            \n            # transform 과정에서 리사이즈된 사진을 원래 크기로 원복함\n            scaled_bboxes = []\n            img_sizes = [(CFG.img_size, CFG.img_size)] * images.shape[0]       \n            \n            \n#             for bboxes, img_dims in zip(boxes, img_sizes):\n                \n#                 im_h, im_w = img_dims\n                \n                \n#                 if len(bboxes) > 0:\n#                     scaled_bboxes.append((np.array(bboxes)*[im_w / CFG.img_size, im_h / CFG.img_size, \n#                                                             im_w / CFG.img_size, im_h / CFG.img_size,]).tolist())

In [11]:
def run_wbf(predictions, image_size=512, iou_thr=0.44, skip_box_thr=0.43, weights=None):
    bboxes = []
    confidences = []
    class_labels = []

    for prediction in predictions:
        boxes = [(prediction["boxes"] / image_size).tolist()]
        scores = [prediction["scores"].tolist()]
        labels = [prediction["classes"].tolist()]
        

        boxes, scores, labels = ensemble_boxes_wbf.weighted_boxes_fusion(
            boxes,
            scores,
            labels,
            weights=weights,
            iou_thr=iou_thr,
            skip_box_thr=skip_box_thr,
        )
        boxes = boxes * (image_size - 1)
        bboxes.append(boxes.tolist())
        confidences.append(scores.tolist())
        class_labels.append(labels.tolist())
        

    return bboxes, confidences, class_labels

# def run_wbf(predictions, image_index, image_size=512, iou_thr=0.44, skip_box_thr=0.43, weights=None):
#     boxes = [(prediction[image_index]['boxes']/(image_size-1)).tolist()  for prediction in predictions]
#     scores = [prediction[image_index]['scores'].tolist()  for prediction in predictions]
#     labels = [np.ones(prediction[image_index]['scores'].shape[0]).tolist() for prediction in predictions]
#     boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
#     boxes = boxes*(image_size-1)
#     return boxes, scores, labels

In [12]:
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        print(j)
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))
    return " ".join(pred_strings)

In [13]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [14]:
def create_dummy_inference_targets(num_images):
    dummy_targets = {
        "bbox": [
            torch.tensor([[0.0, 0.0, 0.0, 0.0]]).to(device)
            for i in range(num_images)
        ],
        "cls": [torch.tensor([1.0]).to(device) for i in range(num_images)],
        "img_size": torch.tensor(
            [(CFG.img_size, CFG.img_size)] * num_images).to(device).float(),
        "img_scale": torch.ones(num_images).to(device).float(),
    }

    return dummy_targets

In [15]:
def test():
    dataset_test = Dataset(data_dir=CFG.data_dir, transform=transform_test())
    loader_test = DataLoader(dataset_test, batch_size=CFG.batch_size, shuffle=False, num_workers=8, drop_last=False, collate_fn=collate_fn)

    net = load_net(CFG.ckpt_dir)
    net = net.to(device)
    
    pred = test_one_epoch(loader_test, net)
    print(pred)

    submission = pd.DataFrame(pred, columns=['image_id', 'PredictionString'])
    submission.to_csv(os.path.join(CFG.result_dir, 'submission.csv'))
    
    return submission


In [24]:
from numbers import Number
from typing import List
from functools import singledispatch

import numpy as np
import torch

from fastcore.dispatch import typedispatch
from pytorch_lightning import LightningModule
from pytorch_lightning.core.decorators import auto_move_data


from ensemble_boxes import ensemble_boxes_wbf


def run_wbf(predictions, image_size=512, iou_thr=0.44, skip_box_thr=0.43, weights=None):
    bboxes = []
    confidences = []
    class_labels = []

    for prediction in predictions:
        boxes = [(prediction["boxes"] / image_size).tolist()]
        scores = [prediction["scores"].tolist()]
        labels = [prediction["classes"].tolist()]

        boxes, scores, labels = ensemble_boxes_wbf.weighted_boxes_fusion(
            boxes,
            scores,
            labels,
            weights=weights,
            iou_thr=iou_thr,
            skip_box_thr=skip_box_thr,
        )
        boxes = boxes * (image_size - 1)
        bboxes.append(boxes.tolist())
        confidences.append(scores.tolist())
        class_labels.append(labels.tolist())

    return bboxes, confidences, class_labels


class EfficientDetModel(LightningModule):
    def __init__(
        self,
        num_classes=1,
        img_size=512,
        prediction_confidence_threshold=0.2,
        learning_rate=0.0002,
        wbf_iou_threshold=0.44,
        model_architecture='tf_efficientnetv2_l',
    ):
        super().__init__()
        self.img_size = img_size
        self.model = create_model(
            num_classes, img_size, architecture=model_architecture
        )
        self.prediction_confidence_threshold = prediction_confidence_threshold
        self.lr = learning_rate
        self.wbf_iou_threshold = wbf_iou_threshold
    
    def forward(self, images, targets):
        return self.model(images, targets)
    
        

In [25]:
net = EfficientDetModel(
    num_classes=1,
    img_size=512
    )


In [ ]:
CFG.ckpt_dir

In [26]:
net.load_state_dict(torch.load(CFG.ckpt_dir))

<All keys matched successfully>

In [27]:
net.eval()

EfficientDetModel(
  (model): DetBenchTrain(
    (model): EfficientDet(
      (backbone): EfficientNetFeatures(
        (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act1): SiLU(inplace=True)
        (blocks): Sequential(
          (0): Sequential(
            (0): ConvBnAct(
              (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
              (act1): SiLU(inplace=True)
            )
            (1): ConvBnAct(
              (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
              (act1): SiLU(inplace=True)
            )
            (2): ConvBnAct(
              (conv

In [28]:
net = net.to(device)

In [29]:
dataset_test = Dataset(data_dir=CFG.data_dir, transform=transform_test())
loader_test = DataLoader(dataset_test, batch_size=CFG.batch_size, shuffle=False, num_workers=8, drop_last=False, collate_fn=collate_fn)

In [30]:
def create_dummy_inference_targets(num_images):
    dummy_targets = {
        "bbox": [
            torch.tensor([[0.0, 0.0, 0.0, 0.0]]).to(device)
            for i in range(num_images)
        ],
        "cls": [torch.tensor([1.0]).to(device) for i in range(num_images)],
        "img_size": torch.tensor([(CFG.img_size, CFG.img_size)] *num_images).to(device).float(),
        "img_scale": torch.ones(num_images).to(device).float(),
    }

    return dummy_targets

In [31]:
net.eval()
pred = []


##
for batch, (images, targets, image_ids) in enumerate(loader_test, 1):
    images = torch.stack(images)
    images = images.to(device).float()
    
    images_tensor = images
    
    if images_tensor.ndim == 3:
        images_tensor = images_tensor.unsqueeze(0)
    if (images_tensor.shape[-1] != 512 or images_tensor.shape[-2] != 512):
        raise ValueError(
            f"Input tensors must be of shape (N, 3, {512}, {512})")

    num_images = images_tensor.shape[0]

    
    # 이건 리스케일시 사용
    image_sizes = [(512, 512)] * num_images

    dummy = create_dummy_inference_targets(num_images)

#     # to(device).flaot()d을 뒤에써야 데이터의 device가 cuda로 간다.
#     boxes = [target['boxes'].to(device).float() for target in targets]
#     img_class = [target['img_class'].to(device).float() for target in targets]
#     img_scale = torch.tensor([target['img_scale'] for target in targets]).to(device).float()
#     img_size = torch.tensor([(512, 512) for target in targets]).to(device).float()

#     target_res = {}
#     target_res['bbox'] = boxes
#     target_res['cls'] = img_class
#     target_res['img_scale'] = img_scale
#     target_res['img_size'] = img_size

In [32]:
with torch.no_grad():
#     print(images)
#     print(dummy_targets)
    det = net(images, dummy)["detections"]
    output = []
 
    # 사진 한장당 루프 한번씩 돌면서 박스 와 스코어와 클래스들을 쪼가 뽑아냄
    for i in range(det.shape[0]):
        boxes = det[i].detach().cpu().numpy()[:,:4]
        #print(len(boxes))
        scores = det[i].detach().cpu().numpy()[:,4]
        #print(len(scores))
        classes = det[i].detach().cpu().numpy()[:, 5]
        #print(classes)
        indexes = np.where(scores > 0.22)[0]
        #print(indexes)
        boxes = boxes[indexes]
        #print(boxes)
        output.append({"boxes": boxes, "scores": scores[indexes], "classes": classes[indexes]}) #  클래스는 1개 밖에 안나왓 ㅓ에러발생
        
    print(output)

[{'boxes': array([[171.80179 , 250.32008 , 221.69025 , 295.48596 ],
       [434.21176 , 265.38513 , 485.70883 , 316.36816 ],
       [141.98126 , 297.57718 , 197.85062 , 349.77554 ],
       [205.33823 , 305.62256 , 261.7181  , 355.7259  ],
       [192.53436 , 245.73029 , 265.78265 , 321.70575 ],
       [198.23582 , 290.43292 , 253.07896 , 340.72504 ],
       [160.58699 , 313.7307  , 212.71977 , 364.1325  ],
       [  9.52739 , 278.27374 , 129.87483 , 357.90198 ],
       [109.22798 , 305.43756 , 168.74284 , 356.59497 ],
       [134.54861 , 316.13657 , 196.1116  , 359.55728 ],
       [183.03351 , 306.47943 , 238.38202 , 356.04224 ],
       [411.95416 , 268.29022 , 468.72742 , 312.43256 ],
       [167.58466 , 299.0354  , 221.88483 , 349.58313 ],
       [176.52032 , 294.5164  , 236.76218 , 338.71292 ],
       [225.54004 , 312.56366 , 278.65567 , 363.97058 ],
       [116.4147  , 289.05362 , 175.02841 , 341.3547  ],
       [ 36.937447, 298.8762  , 112.72103 , 373.49814 ],
       [109.437454, 

In [33]:
boxes, scores, labels = run_wbf(output, image_size=CFG.img_size, iou_thr=0.44)

In [34]:
boxes

[[],
 [],
 [],
 [[8.535664319992065,
   254.32310390472412,
   114.45010708272457,
   335.73288106918335]],
 [],
 [],
 []]

In [35]:
scores

[[], [], [], [0.48094043135643005], [], [], []]

In [36]:
##
if __name__ == "__main__":
    if CFG.mode == "test":
        submission = test()

RuntimeError: Error(s) in loading state_dict for DetBenchTrain:
	Missing key(s) in state_dict: "model.backbone.conv_stem.weight", "model.backbone.bn1.weight", "model.backbone.bn1.bias", "model.backbone.bn1.running_mean", "model.backbone.bn1.running_var", "model.backbone.blocks.0.0.conv.weight", "model.backbone.blocks.0.0.bn1.weight", "model.backbone.blocks.0.0.bn1.bias", "model.backbone.blocks.0.0.bn1.running_mean", "model.backbone.blocks.0.0.bn1.running_var", "model.backbone.blocks.0.1.conv.weight", "model.backbone.blocks.0.1.bn1.weight", "model.backbone.blocks.0.1.bn1.bias", "model.backbone.blocks.0.1.bn1.running_mean", "model.backbone.blocks.0.1.bn1.running_var", "model.backbone.blocks.0.2.conv.weight", "model.backbone.blocks.0.2.bn1.weight", "model.backbone.blocks.0.2.bn1.bias", "model.backbone.blocks.0.2.bn1.running_mean", "model.backbone.blocks.0.2.bn1.running_var", "model.backbone.blocks.0.3.conv.weight", "model.backbone.blocks.0.3.bn1.weight", "model.backbone.blocks.0.3.bn1.bia

In [ ]:
submission

In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
# visualize a test image

def make_predictions(images, net, score_threshold=0.22):
    images = torch.stack(images).cuda().float()
    predictions = []
    with torch.no_grad():
        det = net(images, torch.tensor([1]*images.shape[0]).float().cuda())
        for i in range(images.shape[0]):
            boxes = det[i].detach().cpu().numpy()[:,:4]    
            scores = det[i].detach().cpu().numpy()[:,4]
            indexes = np.where(scores > score_threshold)[0]
            boxes = boxes[indexes]
            boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
            boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
            predictions.append({
                'boxes': boxes[indexes],
                'scores': scores[indexes],
            })
    return [predictions]

In [ ]:
dataset_test = Dataset(data_dir=CFG.data_dir, transform=transform_test())
loader_test = DataLoader(dataset_test, batch_size=CFG.batch_size, shuffle=False, num_workers=8, drop_last=False, collate_fn=collate_fn, pin_memory=True)
net = load_net(CFG.ckpt_dir)

for j, (images, image_ids) in enumerate(loader_test):
    break

print(image_ids)

In [ ]:
predictions = make_predictions(images, net)

# choose test image idx
i = 3
sample = images[i].permute(1,2,0).cpu().numpy()

boxes, scores, labels = run_wbf(predictions, image_index=i)
boxes = boxes.astype(np.int32).clip(min=0, max=511)

for box in boxes:
    cv2.rectangle(sample, (box[0], box[1]), (box[2], box[3]), (1, 0, 0), 1)
    
cv2.imwrite('sample.jpg', sample)

In [ ]:
predictions = make_predictions(images, net)

# choose test image idx
i = 3
sample = images[i].permute(1,2,0).cpu().numpy()

boxes, scores, labels = run_wbf(predictions, image_index=i)
boxes = boxes.astype(np.int32).clip(min=0, max=511)

fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    cv2.rectangle(sample, (box[0], box[1]), (box[2], box[3]), (1, 0, 0), 1)
    
ax.set_axis_off()
ax.imshow(sample);

In [ ]:
plt.imsave('thiisample.jpg', sample)

In [ ]:
sample

In [ ]:
from numbers import Number
from typing import List
from functools import singledispatch

import numpy as np
import torch

from fastcore.dispatch import typedispatch
from pytorch_lightning import LightningModule
from pytorch_lightning.core.decorators import auto_move_data


from ensemble_boxes import ensemble_boxes_wbf


class EfficientDetModel(LightningModule):
    def __init__(
        self,
        num_classes=1,
        img_size=512,
        prediction_confidence_threshold=0.2,
        learning_rate=0.0002,
        wbf_iou_threshold=0.44,
        model_architecture='tf_efficientnetv2_l',
    ):
        super().__init__()
        self.img_size = img_size
        self.model = create_model(
            num_classes, img_size, architecture=model_architecture
        )
        self.prediction_confidence_threshold = prediction_confidence_threshold
        self.lr = learning_rate
        self.wbf_iou_threshold = wbf_iou_threshold
        

In [ ]:
net = EfficientDetModel(
    num_classes=1,
    img_size=512
    )


In [ ]:
net.load_state_dict(torch.load(CFG.ckpt_dir))